
<div style="border:1px solid black; padding:20px 20px;text-align: justify;text-justify: inter-word">
    <strong>Project of basics of mobile robotics - Thymio in Paris<br/> Autumn 2021 <br/> </strong><br/>

<div style="justify;text-justify: inter-word">
Bonvalot Isione <br>
Franca Sébastien <br>
In-Albon Malika <br>
Pellegrini Sylvain <br>
</div>



---
# Introduction 


## Context
<p align="justify"> The basics of mobile robotics' project consists in a visit of Paris. Thymio is on holiday in Paris. He had a long day, so he intends to stroll down the Champs Elysées and under the Arc of Triomphe by the shortest path through traffic. Then, Thymio wants to enjoy a view of the Eiffel Tower, still by the shortest path. During his walk, some Parisians in a hurry crosses his road but thanks to his good ability, Thymio avoids them and continues along the streets. </p>

## Environment set-up

### Environment
<p> The ground where Thymio navigates is white.  parler de la dimension de notre set up ?</p>

### Goal
<p> The project's set-up is composed of a goal positions represented by a green circle. Parler du diamètre ? </p>

### Robot 
<p> The robot is recognizable by two blue circles on top of it. The two circles have different sizes in order to know the direction of Thymio (small circle on the front and big circle on the back of the robot). parler du diamètre ?  </p>

### Permanent obstacles

#### Arc of Thriomphe
<p> The Arc of Triomphe is a 3D rectangle obstacle in red with brown sides. Thymio should pass under this obstacle. </p>

#### Other obstacles
<p> The other obstacles are in 2D and are black top. They have different shapes : triangles and squares. </p>

### Unforeseen obstacles 
<p> The unforeseen obstacles is a 3D white cylinder. </p>

### Camera
<p> The camera used for the project is the camera provided in class. It is ....parler du modèle ? . The camera is hung to a lamp on top of the set-up.</p>


---
# Choices

---
## Vision

### Position

---
## Navigation

### Global navigation

**For simplicity, the lists' indices used in the Global Navigation module follow this convention: <br>
index 0 corresponds to start, maximal index corresponds to goal, and all intermediary indices correspond to obstacle vertices. <br>
This allows us to work with points' indices (also referred to as names) rather than points' coordinates.**
<br/>

#### Visibility graph
The code for computing the visibility graph is given in global_path.py.<br/>

It takes as input:
- start, the start position as tuple (x,y)
- goal, the goal position as tuple (x,y)
- list_vertices, the position of every obstacle's vertices grouped by obstacle: <br>
[[(x1.1,y1.1),(x1.2,y1.2),...,(x1.V1,y1.V1)],...,[(xO.1,yO.1),(xO.2,yO.2),...,(xO.VO,yO.VO)]] <br>
where Vi is the number of vertices of obstacle i, and O is the total number of obstacles. <br/>

The output is list_neighbours = <br>
[[neighbours_of_start], [neighbours_of_vertex1],..., [neighbours_of_goal]] <br>
where neighbour_of_XXX is list of tuples which each contain the index and distance of neighbours: (index_neighbour, distance_neighbour)

<br/>

##### Initialization

<br/>
The initialization consists in two steps :<br>

1. Listing all points on the graph. <br>
In this step, the coordinates of every obstacle's vertices, as well as start's and goal's coordinates, are read and stored together in a list. <br>
Also, all coordinates are transformed into instances of class check_intersection.Point. This will allow us to use the function check_intersection.doIntersect() more easily.<br>
*The code check_intersection.py was taken from https://www.geeksforgeeks.org/check-if-two-given-line-segments-intersect/ (cons. 27.11.2021) and was developed by Ansh Riyal.*


2. Listing all obstacle sides.<br>
In this step, we want to construct each obstacle as a series of vertex indices. For example, a triangle could be stored as [(1,2),(2,3), (3,1)]. <br>
To do so, we start by counting how many vertices (i.e. how many sides) each obstacle has. <br>
Then, for each obstacle, we build a list of its sides. To do so, we first need to calculate a shift k from which to start giving vertices' indices. For example, for an obstacle whose first vertex's name is k and which has v vertices, the result would be: [(k,k+1), (k+1,k+2), ..., (k+(v-1), k)]. <br>
Finally, we store all lists of sides into a bigger list.

<br/>
The codes for initialization are given in the following cells.

In [ ]:
def flatten_list_points(list_vertices, start, goal):
    """
    Returns a "flat" (i.e. list without sublists or tuples) version of
    all point coordinates including start and goal.
    Each entry is an instance of class check_intersection.Point
    """    
    list_points = []         
    for obstacle in range(len(list_vertices)):
        for vertex in range(len(list_vertices[obstacle])):
            x = list_vertices[obstacle][vertex][0]
            y = list_vertices[obstacle][vertex][1]
            list_points.append(check_intersection.Point(x,y))                   #insert obstacle vertices
                                                            
    list_points.insert(0, check_intersection.Point(start[0],start[1]))          #insert start
    list_points.append(check_intersection.Point(goal[0],goal[1]))               #insert goal
    
    return list_points

In [ ]:
def obtain_list_sides(list_vertices):
    """
    Given the list of obstacle vertices sorted by obstacle,
    this function returns the list of obstacle sides sorted by obstacle.
    """   
    #step 1: obtain the number of sides of each obstacle
    length_obstacles = [len(obstacle) for obstacle in list_vertices]
        
    #step 2: convert indexing of vertices from [0][0],[0][1],... into 1,2,3,...,N 
    #by introducing a shift (0 and N+1 are start and goal respectively)
    
    #then enter every tuple (side.vertex1, side.vertex2) into sides_obstacle
    #then enter sides_obstacle into list_sides
    list_sides = []
    for obstacle in range(len(list_vertices)):
        sides_obstacle = []                             #list of sides of current obstacle
        
        shift = sum(length_obstacles[0:obstacle]) + 1   #to convert indexing
        nb_sides = length_obstacles[obstacle]
        for vertex in range(nb_sides):
            side = (shift + vertex,\
                    shift + ((vertex+1) % nb_sides))    #use of % to count 1,2,1 instead of 1,2,3
            sides_obstacle.append(side)
        
        list_sides.append(sides_obstacle)               #total list of sides, grouped by obstacle

    return list_sides

##### Completing the list of neighbours
<br/>

We now want to indicate for each point:
- who its neighbours are
- how far they are from that point<br>

*(Note that the term "neighbour" refers to all points that are visible by the current point, i.e. no line connecting two neighbours passes trough an obstacle.)*
<br/><br/>

To simplify our search, we assume that our obstacles are simple shapes such as squares or triangles. Thanks to this assumption, we only have five different possibilities to determine whether two points p1,p2 are neighbours (their connection line **CL** is valid) or not: <br>

1. (p1,p2) is a side of an obstacle $\rightarrow$ CL is valid (segment v1 on the following figure)<br>
2. p1 and p2 lie on the same obstacle, but CL is not a side of the obstacle $\rightarrow$ CL is not valid (segment nv1 on the following figure)
3. p1 and p2 do not lie on the same obstacle. p1 and/or p2 lie on all lines that CL intersects $\rightarrow$ CL is valid (segment v2 on the following figure)
4. p1 and p2 do not lie on the same obstacle. CL intersects some lines on which neither p1 nor p2 lie $\rightarrow$ CL is not valid (segment nv2 on the following figure)
5. Other cases: we consider that CL is valid (this did not cause any problems upon testing).

Note that this logic is not robust to complicated obstacle shapes. On the following figure, nv3 would not be valid due to condition 2. However, this logic is sufficient with the kind of obstacles that we use.<br>
<br/>


##### Overlaps
<br/>

The obstacles used for the algorithm are enlarged versions of the real obstacle, to allow Thymio to pass in their neighbourhood without hitting them. This means that there is a risk of overlap between two obstacles.<br>
To take it into account, we create a list with all obstacle vertices located within another obstacle (e.g. point 9 on the following figure). If a point is within this list, we will bypass the above conditions and force it to have no neighbours.



![Visibility](./images/VisibilityGraph.JPG) <br/>

Note the indexing convention: point 0 is the start, point 15 is the goal, all intermediary indices are obstacles' vertices. <br/>

Our algorithm considers that v1 and v2 are valid (conditions 1 and 3 respectively). <br>
nv1,nv2,nv3 are not valid (conditions 2, 4, 2 respectively).

The situation for nv4 is a bit more complex. It is a side of obstacle 12-13-14, hence considered valid at first (condition 1). But once we check nv4 with obstacle 9-10-11, it is considered invalid (condition 4). In conclusion, nv4 is considered not valid.

Now that these conditions are set, here is the pseudo-code to explain the logic on one point and one obstacle:

And now, the python implementation for all points and obstacles: 

In [1]:
import point_in_polygon as pt_in_pgn #credits: #Vikas Chitturi, https://www.geeksforgeeks.org/check-if-two-given-line-segments-intersect/  (cons. 09.12.2021)

def find_overlaps(list_vertices):
    #find the length of each obstacle (to introduce shift)
    length_obstacles = [len(obstacle) for obstacle in list_vertices]
    
    #initialize the return
    list_overlaps = []
    
    #check for each vertex whether it lies inside another obstacle than its own
    for obstacle in range(len(list_vertices)):
        vertices_other_obstacles = list_vertices.copy()
        vertices_other_obstacles.remove(list_vertices[obstacle])
        
        shift = sum(length_obstacles[0:obstacle]) + 1   #count vertices k,k+1,... instead of 0,1,...
        
        for other_obstacle in vertices_other_obstacles:
            for vertex_idx_in_obstacle in range(len(list_vertices[obstacle])):
                name_vertex = shift + vertex_idx_in_obstacle
                coords_vertex = list_vertices[obstacle][vertex_idx_in_obstacle]
                if pt_in_pgn.is_inside_polygon(other_obstacle, coords_vertex):
                    list_overlaps.append(name_vertex)
    
    return list_overlaps


def find_all_paths(list_vertices, start, goal):
    # 1. Initialisation
    
    #transform list_vertices into a list of all points including start & goal. 
    #store them without any distinction between obstacles/start/goal
    #every entry is an instance of class Point (see check_intersection.py)
    list_points = flatten_list_points(list_vertices, start, goal)
    nb_vertices = len(list_points) - 2
    
    #extract list_sides, the list of all obstacle sides e.g. (vertex1,vertex2),(vertex2,vertex3),etc
    #list_sides is grouped by obstacle:  list_sides = [[obstacle1.sides], [obstacle2.sides], ...]
    #also, find all overlapping obstacles
    list_sides = obtain_list_sides(list_vertices) 
    list_overlaps = find_overlaps(list_vertices)

    # 2. Search algorithm
    # Determine for each point, what other points it can "see"
    # or in other terms, what connections are considered as valid    
    
    # There are 5 possibilities with our type of obstacles, implemented in code:
        # 1. (point1,point2) is a side of the obstacle --> valid connection
        # 2. both points lie on the obstacle, but (point1,point2) is not a side of the obstacle --> not valid
        # 3. 1st & 2nd condition false, and at least one of the points is equal to a vertex being tested --> valid
        # 4. other conditions false, and (point1,point2) intersects (vertex1, vertex2) --> not valid
        # 5. after testing, none of these conditions are fulfilled --> valid
    
    dist = lambda x1,y1,x2,y2: math.sqrt((x1-x2)**2 + (y1-y2)**2)
    
    list_neighbours = []                        #final output: list of neighbours of all points
    for point1_name in range(nb_vertices+2):
        ngb_current_point = []                  #list of neighbours of the current point
        
        for point2_name in range(nb_vertices+2):
            if point1_name==point2_name:        #do not check self-connection
                pass
            
            elif (point1_name in list_overlaps) or (point2_name in list_overlaps): #do not check overlapping obstacles
                pass
            
            else:
                point1 = list_points[point1_name]
                point2 = list_points[point2_name]
                valid_connection = True         #valid until proven wrong
                
                #Test the 5 conditions listed above
                for obstacle in list_sides:
                    obstacle_flattened = [vertex for side in obstacle for vertex in side]
                    
                    
                    condition1 = (((point1_name,point2_name) in obstacle)\
                                  or ((point2_name,point1_name) in obstacle))
                        
                    condition2 = ((point1_name in obstacle_flattened)\
                                  and (point2_name in obstacle_flattened))
                    
                    if condition1:     
                        pass
                        
                    elif condition2:        
                        valid_connection = False                        
                        
                    else:
                        for side_vertices in obstacle:
                            vertex1_name = side_vertices[0]
                            vertex2_name = side_vertices[1]
                            vertex1 = list_points[vertex1_name]
                            vertex2 = list_points[vertex2_name]
                            
                            
                            condition3 = ((point1_name==vertex1_name) or (point2_name==vertex2_name)\
                                or (point2_name==vertex1_name) or (point1_name==vertex2_name))
                            
                            condition4 = check_intersection.doIntersect(point1, point2, vertex1, vertex2)
                                
                            
                            if condition3:         
                                    pass
                                    
                            elif condition4:          
                                    valid_connection = False
                               
                            #else: condition5=True ; pass
                          
                
                
                if valid_connection:
                    ngb_current_point.append((point2_name, dist(point1.x, point1.y, point2.x, point2.y)))
        
        list_neighbours.append(ngb_current_point)
        
    return list_neighbours

#### Optimal path finding

Now that the visibility graph is known, we can explore it to determine which path leads to the goal in an optimal way. This relies on the A* algorithm (the logic and pseudo-codes used in this section are inspired from the lecture slides and the Wikipedia page on A* algorithm: https://en.wikipedia.org/wiki/A*_search_algorithm, *cons. 08.12.2021*). <br/>

The code for finding the optimal path is given in shortest_path.py. Its most relevant function that is called in main code is find_shortest_path(). <br>
It takes as input:
- obstacle_vertices, which is the same vector used for building the visibility graph
- start and goal position, as above <br>

It returns a list: names_full_path = [0, v1, v2, ..., vP, P+1] where each entry is the index of a point. Again, 0 correspond to start ; v1,v2,...,vP is the P-steps path to follow ; and P+1 is the goal.<br/>

We coded the module in a way that it can apply either A* (with Euclidean distance from goal as a heuristic) or Dijkstra (with a heuristic=0 for all points). To use Dijkstra, the function must be called without specifying the start and goal positions. <br>
After testing, we decided to use A* for the final implementation as it seemed to yield better results than Dijkstra. <br/><br/>

##### Initialization

The initialization follows this order:<br>
1. Calculate the visibility graph thanks to global_path.py
2. Create a list with all points' names: [0, 1, ..., N, N+1] following the convention 0$\rightarrow$start and N+1$\rightarrow$goal
3. Assign the heuristic to every point of the graph
4. Attribute infinite weight to every point of the graph, except the start (weight_start = 0 + heuristic)
5. Initialize the following lists: 
    - predecessors, where we will store pieces of the optimal path before reconstructing it entirely. For example: "To reach goal, come from vertex 5. To reach vertex 5, come from vertex 2. To reach vertex 2...".
    - vertices_to_explore, which will indicate what points on the graph are still unexplored. <br/>

The code for heuristic attribution is given below.


In [ ]:
def get_heuristics(N, obstacle_vertices_flattened, start, goal):
    """
    Assigns a heuristic to each point. 
    If the start and goal are not specified, then standard Dijkstra is applied (heuristic=0).
    Otherwise, the heuristic is the Euclidean distance to the goal.
    
    Inputs: 
        - goal, start: identical to ones given to find_shortest_path
        - N: total number of obstacle vertices (obtained from find_shortest_path)
        - obstacle_vertices_flattened: list of obstacles vertices 
        NOT sorted by obstacle (obtained from find_shortest_path)
        
    Outputs: heuristic for each point [h0, h1, ..., hN, 0]
    """
    heuristic = [0]*(N+2)
    dist = lambda p1,p2: math.sqrt((p1[1]-p2[1])**2+(p1[0]-p2[0])**2)
    if goal!=("unspecified","unspecified"):
        heuristic = [dist(goal,vertex) for vertex in obstacle_vertices_flattened]
        heuristic.append(0)                      #dist_goal2goal = 0
        heuristic.insert(0,dist(goal,start))     #dist_start2goal
        
    return heuristic

##### Graph exploration <br/>

The pseudo-code for graph exploration is given in the following cell. A few precisions:
- current_neighbours contains the names of current_point's neighbours, as well as their distance from current_point
- predecessors is the list where pieces of the optimal path are stored for later reconstruction

The Python implementation for this pseudo-code is given in the next cell.

In [ ]:
def explore_graph(vertices_to_explore, name_vertices, list_neighbours, weights_total, weights_Dijkstra, heuristic, predecessors, goal):
    """" This function applies the Dijkstra (or A*) algorithm:
        at each step, it chooses the point with smallest weight
        and explores from there.
        It does so until reaching the goal (case A*) 
        or until the whole graph is explored (case Dijkstra)
        
        Inputs: 
            - list_vertices, goal: identical to ones given to find_shortest_path
            - other inputs: taken directly from find_shortest_path
            
        Output: 
            predecessors, list of ideal predecessor for each node [p0,p1,...,pN],
            where each entry is the name of a node
    """
    #the while loop stops when vertices_to_explore is empty
    while vertices_to_explore:                                
        weights_unexplored = []
        for vertex in vertices_to_explore:
            weights_unexplored.append(weights_total[vertex])
        
        #set the current vertex to the one with lowest weight. 
        #get its neighbours' names and their associated distance (name,dist)
        current_vertex = vertices_to_explore[weights_unexplored.index(min(weights_unexplored))]
        current_neighbours = list_neighbours[current_vertex]
        
        #for each neighbour of the current vertex, calculate its new total weight.
        for ngb in current_neighbours:
            ngb_name = ngb[0]
            ngb_dist = ngb[1]
            
            weight_candidate = weights_Dijkstra[current_vertex] + ngb_dist + heuristic[ngb_name]
            
            #if this weight is better than previous, 
            #update it and set current_vertex as best predecessor for this neighbour.
            if weight_candidate<weights_total[ngb_name]:
                weights_total[ngb_name] = weight_candidate
                weights_Dijkstra[ngb_name] = weights_Dijkstra[current_vertex] + ngb_dist
                predecessors[ngb_name] = current_vertex
                
                #if this neighbour was already explored, allow to re-explore it to update weights
                if ngb_name not in vertices_to_explore:
                    vertices_to_explore.append(ngb_name)
        
        #remove the current vertex from unexplored vertices
        vertices_to_explore.remove(current_vertex)
        
        
        
        #if A* is used, exit the loop once the goal is reached (no need to explore the entire graph)
        current_neighbours_flat = [x for sublist in current_neighbours for x in sublist]
        name_current_neighbours = current_neighbours_flat[0::2]                 #name of neighbours
        
        if goal!=("unspecified","unspecified")\
            and (name_vertices[-1] in name_current_neighbours):                
            #print("Unexplored nodes: {}".format(vertices_to_explore)) #some nodes are unexplored
            break
        
        
    
    return predecessors

##### Reconstructing the path

Once the list *predecessors* is filled, we work backwards to reconstruct the entire path until the start is reached. <br>
For example, let's say predecessors = ["reached_start", 0, 0, 2, 1, 3]. The goal (index 5) is reached from index 3. Index 3 is reached from index 2. Index 2 is reached from index 0 (start). <br>
The optimal path is then: [0, 2, 3, 5].

##### Main code for global navigation
Finally, we give below the code which implements the whole global navigation, following the logic described here and using the functions defined above.

In [ ]:
import math
import global_path

def find_shortest_path(obstacle_vertices, start=("unspecified","unspecified"), goal=("unspecified","unspecified")):
    # 1. Initialisation
    #find all neighbours
    list_neighbours = global_path.find_all_paths(obstacle_vertices, start, goal)

    #attribute a name to each point: start (point 0), obstacle vertices (vertices 1...N), and goal (point N+1)
    obstacle_vertices_flattened = [item for sublist in obstacle_vertices for item in sublist]
    N = len(obstacle_vertices_flattened)                                            
    name_vertices = list(range(N+2))

    #attribute heuristics to each vertex (Euclidean distance if A*, 0 otherwise)
    heuristic = get_heuristics(N, obstacle_vertices_flattened, start, goal)

    #attribute initial weights to each vertex
    INFINITY = 100000
    weights_Dijkstra = [INFINITY]*(N+2)      # min distance to reach each vertex from start. Before exploration, =infinity
    weights_total = weights_Dijkstra.copy()  # weights_total = weights_Dijkstra + heuristic
    weights_Dijkstra[0] = 0
    weights_total[0] = heuristic[0]
    
    #initialize vertex of predecessors, used to trace back the best path
    predecessors = ["unknown"]*(N+2)
    predecessors[0] = "reached_start"
    
    #list of vertices to be explored
    vertices_to_explore = name_vertices.copy()
    vertices_to_explore.remove(N+1)          #no need to explore the goal
    
    
    
    # 2. Apply Dijkstra or A*
    explore_graph(vertices_to_explore, name_vertices, list_neighbours,\
                  weights_total, weights_Dijkstra, heuristic,\
                      predecessors, goal)
    
        
        
    # 3. Retrace the best path going backwards 
    #(origin = where we came from. predecessor = best origin for a given point)
    names_full_path = []
    origin = predecessors[-1]                      #predecessor of goal
    while origin!="reached_start":
        print(origin)
        names_full_path.insert(0,origin)           #insert predecessor of vertex i
        origin = predecessors[origin]              #for next loop: change the value of "origin"
    names_full_path.append(name_vertices[-1])      #add vertex "goal" to the names_full_path


    return names_full_path

##### Auxiliary code: conversion from points' names to points' coordinates

In [ ]:
def pathname_to_coords(names_path, list_vertices, start, goal):
    """"Converts the return of shortest_path into coordinates.
    
    Inputs:
        similar as previous functions. names_path is the return from shortest_path.
        
    Output:
        - coords = [(node0.x, node0.y), (nodeK.x,nodeK.y),..., (nodeP.x, nodeP.y), (node(N+1).x, node(N+1).y)]
        where the robot must do node0 --> nodeK --> ...--> nodeP --> node(N+1)
        """
    obstacle_vertices_flattened = [vertex for sublist in list_vertices for vertex in sublist]

    coords = [start]
    for i in names_path[1:-1]:
        coords.append(obstacle_vertices_flattened[i - 1])
    coords.append(goal)

    return coords

### Local navigation </br>

<p> <i> The code was inspired be the exercises sessions 3 and 4 from the course MICRO-452 : Basics of mobile robotics. </i> </p>

<p> The local navigation is used to avoid physical obstacles that are unpredictible. These obstacles can be put in Thymio's path at any point in time so that the Thymio should leave the optimal path to avoid them. Once the obstacle is passed, the robot has to go back to the initial optimal path. This is done by the motion control module. There are many possibilities to implement a local avoidance using the proximity sensors. </p> </br>

#### Kind of ANN (trouver un titre stylé svp) </br>

##### Obstacle detection
<p>To detect an obstacle, the five front proximity sensors are used. --> mettre un dessin du robot avec les capteurs de prox.</p>
<ul>
    <li> To detect an obstacle on the left side : we calculate the mean of the first and second sensors.</li>
    <li> To detect an obstacle on the right side : we calculate the mean of the fourth and fifth sensors. </li>
    <li> To detect an obstacle in front of the robot : we measure the value of the third sensors.</li>
</ul>
</br>

<p> The code to check if an obstacle is detected is given below. </p>

In [ ]:
OBST_THR_L = 10      # low obstacle threshold 
OBST_THR_H = 60      # high obstacle threshold 

def check_obstacle(prox_sensors):
    # acquisition from the proximity sensors to detect obstacles
    obst = [prox_sensors[0], prox_sensors[1], \
            prox_sensors[2], prox_sensors[3], \
            prox_sensors[4]]
    mean_obst_left = (obst[0] + obst[1])//2
    mean_obst_right = (obst[3] + obst[4])//2
    obst_front = obst[2]
    
    if (mean_obst_right > OBST_THR_H) or (mean_obst_left > OBST_THR_H) \
        or (obst_front > OBST_THR_H) :
        obstacle = True
    else:
        obstacle = False
    return obstacle, obst_front, mean_obst_left, mean_obst_right


##### Pseudo-code
 <p> The following pseudo-code describes the different steps once an obstacle is detected somewhere in front of the robot.  </p>
<p> There are two cases depending on the position of the unforseen obstacle. 
<ul>
<li>Case 1 : Obstacle only in front of the robot (detected only by the third sensor) </li>
    <ul>
    <li> The robot turns in place to the left (tester de randomiser la direction qu'il prend ?</li>
    <li> The speed is constant </li>
    </ul>
<li>Case 2 : Obstacle detected on either the left or the right side</li>
    <ul>
    <li> The robot accelerates in the opposite direction of the obstacle</li>
    <li> The speed depends on the values of the sensors means </li>
    </ul>
</ul>  

For the first case, to ensure that the robot moves straight forward to the obstacle, we check that the values of the wheels are almost the same. (tournure de phrase pas ouf je trouve)
</p>

The code concerning the case 1 is given below.

In [ ]:
if (abs(motor_left_speed - motor_right_speed) < 50):
    if obst_front > OBST_THR_H:
        motor_left = -SPEED0
        motor_right = SPEED0
    else:
        obst_in_front = 0

The code concerning the case 2 is given below.

In [ ]:
if obst_in_front == 0:
    motor_left = SPEED0 + OBST_SPEED_GAIN*( mean_obst_left//100)
    motor_right = SPEED0 + OBST_SPEED_GAIN*( mean_obst_right//100)

---
## Motion Control

### Proportional controller 

---
## Filtering

### Bayesian filtering - Kalman filter

---
## Finite state machine

-> est ce qu'on la mettrait pas plutôt au tout début en 1ere sou

---
# Main code to execute


In [ ]:
from tdmclient import ClientAsync, aw
client = ClientAsync()
node = await client.wait_for_node()
await node.lock()
await node.wait_for_variables()

In [ ]:
import vision as vis
import kalman
import numpy as np
import time
import cv2 as cv
import shortest_path as short
import positions as pos
import math
import motion_control as motion

COMPUTE = 0
TRAVEL = 1
LOCAL_NAV = 2
FINISH = 3

SPEED0 = 50       # nominal speed
OBST_SPEED_GAIN = 10  # /100 (actual gain: 5/100=0.05)
SENSOR_SCALE = 200  # Scale factors for sensors

OBST_THR_L = 10      # low obstacle threshold 
OBST_THR_H = 60      # high obstacle threshold 

state = 0          # 0= go to goal, 1=obstacle avoidance
obst = [0,0,0,0,0]   # measurements from the 2 left and the 2 right prox sensors

statego2goal = 0
stateobstavoid = 1

In [ ]:
# Main -> state machine
state = COMPUTE
obstacle = False
path_calculated = False

#valeurs pour initialiser
robot_pos_kalman = np.array([[0],[0],[0]]) #3-by-1 np.array
Sigma_kalman = np.diag([0.01,0.01,0.01])
HISTORY_SIZE = 15
history_kalman = [np.reshape(robot_pos_kalman,(1,3))]*HISTORY_SIZE

cap = cv.VideoCapture(1)  
cap.set(cv.CAP_PROP_AUTO_EXPOSURE, 1)
while True:
    if state == COMPUTE:
        captured, img = cap.read()
        if captured:
            cropped, cropped_img = vis.crop_map(img)
            # print(cropped_img.shape)        # uncomment to debug
            # cv.imshow("Original", img)        # uncomment to debug
            cv.imshow("Cropped", cropped_img)
            if cropped:
                im3 = np.zeros(cropped_img.shape)
                found, obstacles = vis.extract_obstacles(cropped_img)
                if found:
                    ex_obstacles = vis.expand_obstacles(obstacles)
                    cv.drawContours(im3, ex_obstacles, -1, (0, 255, 0), 3)
                    cv.imshow("expanded", im3)
                found_rob, robot_pos = pos.get_robot_position(cropped_img)
                if found_rob:
                    vis.annotate_robot(robot_pos, im3)
                found_goal, goal_pos, radius_pxl = pos.get_goal_position(cropped_img)
                if found_goal:
                    vis.annotate_goal(goal_pos, im3)
                found_arch, arch_pos = pos.get_arch_positions(cropped_img)
                if found_arch and found_rob:
                    if math.dist(arch_pos[0],robot_pos[0])>math.dist(arch_pos[1],robot_pos[0]):
                        arch_pos = np.flip(arch_pos,0)
                    vis.annotate_arch(arch_pos, im3)
                if (found and found_rob and found_goal and found_arch):
                    scale_factor = pos.convert_meter2pxl(radius_pxl)
                    AXLE_LENGTH = 0.095*scale_factor #used for kalman filter
                    if path_calculated == False:
                        vertices = vis.convert_vertice(ex_obstacles)
                        pathname = short.find_shortest_path(vertices, tuple(robot_pos[0]), tuple(arch_pos[0]))
                        path_arch =\
                        short.pathname_to_coords(pathname,vertices,tuple(robot_pos[0]), tuple(arch_pos[0]))
                        vis.annotate_path(path_arch,im3)

                        pathname2 = short.find_shortest_path(vertices, tuple(arch_pos[1]), tuple(goal_pos))
                        path_goal =\
                        short.pathname_to_coords(pathname2,vertices,tuple(arch_pos[1]), tuple(goal_pos))
                        vis.annotate_path(path_goal,im3)
                        cv.imshow("environment", im3)
                        
                        path = []
                        path.extend(path_arch)
                        path.extend(path_goal)
                                                
                        #initialize kalman filter
                        robot_pos_kalman = np.array([[robot_pos[0][0]],[robot_pos[0][1]],[robot_pos[1]]]) #3-by-1 np.array
                        HISTORY_SIZE = 10
                        history_kalman = [np.reshape(robot_pos_kalman,(1,3))]*HISTORY_SIZE
                        Ts = 0.15 #initialize
                        
                        state = TRAVEL
        else:
            print("There was a problem in the capture")
            break
    if state == TRAVEL:
        cv.imshow("environment", im3)
        cv.waitKey(0)
        
        for subgoal in path:
            while not motion.check_robot_arrived(robot_pos,subgoal):
                prox_sensors = np.array([x for x in node['prox.horizontal'][0:5]])
                print(prox_sensors)
                obstacle, obst_front, mean_obst_left, mean_obst_right = motion.check_obstacle(prox_sensors)
                
                if obstacle == True:
                    # obstacle avoidance: accelerate wheel near obstacle
                    captured, img_local = cap.read()
                    success_local, robot_pos_local = pos.get_robot_position(img_local)
                    im4 = np.zeros(img_local.shape,np.uint8)

                    if success_local:
                        vis.annotate_robot(robot_pos_local, im4)
                        
                    vis.annotate_path(path,im4)
                    cv.putText(im4, "Local avoidance", (40, 40), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                    cv.imshow("mission status", im4)
                    cv.waitKey(10)
                    obst_in_front = 1
                    
                    await node.wait_for_variables({"motor.left.speed"})
                    await node.wait_for_variables({"motor.right.speed"})
                    motor_left_speed = node.v.motor.left.speed
                    motor_right_speed = node.v.motor.right.speed
                    if (abs(motor_left_speed - motor_right_speed) < 50):
                        if obst_front > OBST_THR_H:
                            motor_left = -SPEED0
                            motor_right = SPEED0
                        else:
                            obst_in_front = 0

                    if obst_in_front == 0:
                        motor_left = SPEED0 + OBST_SPEED_GAIN*(mean_obst_left//100)
                        motor_right = SPEED0 + OBST_SPEED_GAIN*(mean_obst_right//100)

                else: #if obstacle == False
                    
                    captured, img_travel = cap.read()
                    cropped, cropped_travel = vis.crop_map(img_travel)
                    cv.imshow("cropped",cropped_travel)
                    im4 = np.zeros(cropped_travel.shape)
                    vis.annotate_robot(robot_pos,im4)
                    vis.annotate_path(path,im4)
                    cv.imshow("mission status", im4)
                    cv.waitKey(10)

                    #read values from camera & sensors
                    success_camera, robot_pos_camera = pos.get_robot_position(cropped_travel)
                    #print("robot angle", robot_pos_camera[1])
                    if success_camera:
                        camera_input = [(robot_pos_camera[0][0],robot_pos_camera[0][1]), robot_pos_camera[1], success_camera]
                        print("seen by camera", camera_input)
                    else:
                        camera_input = [(0,0),0,False]
                    await node.wait_for_variables({"motor.left.speed"})
                    await node.wait_for_variables({"motor.right.speed"})
                    motorspeed = [node.v.motor.right.speed, node.v.motor.left.speed]        


                    #get position from kalman filter
                    robot_pos_kalman, Sigma_kalman, history_kalman =\
                    kalman.kalmanfilter(robot_pos_kalman,Sigma_kalman,motorspeed,history_kalman, camera_input, Ts, scale_factor, AXLE_LENGTH)
                    robot_pos = [(int(robot_pos_kalman[0][0]), int(robot_pos_kalman[1][0])), robot_pos_kalman[2][0]]
                    print("robot_pos",robot_pos)
                    print("kalman[0][0]", robot_pos_kalman[0][0])
                    print("kalman[1][0]", robot_pos_kalman[1][0])
                    print("kalman[2][0]", robot_pos_kalman[2][0])
                    #et apres, on passe robot_pos au controleur


                    err = motion.get_error(robot_pos,subgoal)
                    cv.putText(im4, str(err), (40, 40), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    cv.imshow("mission status", im4)
                    cv.waitKey(10)
                    motor_left, motor_right = motion.speed_control(err)
                    motor_left = round(motor_left)
                    motor_right = round(motor_right)
                    
                v = {
                    "motor.left.target": [motor_left],
                    "motor.right.target": [motor_right],
                }
                aw(node.set_variables(v))
        state = FINISH
    if state == FINISH:
        v = {
                    "motor.left.target": [0],
                    "motor.right.target": [0],
                }
        aw(node.set_variables(v))
            #print("motor_left",motor_left)
            #print("motor_right", motor_right)
            #motor_left_target = int(motor_left)
            #motor_right_target = int(motor_right)
            #print("motor_left_t",motor_left_target)
            #print("motor_right_t", motor_right_target)          
    cv.imshow("environment",im3)
    #cv.waitKey(0)
cap.release()
print("finished!")
# cv.destroyAllWindows()
  
    